In [5]:
#from google.colab import drive
#drive.mount('/content/drive')

In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from keras.models import Sequential
from keras.layers import Dense, Dropout
from google.colab import files

# Redes Neurais

Hiperparâmetros:

- loss: função de perda para avaliação do erro. ex: mean_squared_error
- optimizer: otimizador de ajuste de pesos da rede. stochastic gradient descnet
- metrics: a métrica que a rede vai utilizar para avaliar a performance


In [7]:
spam = pd.read_csv("spam.csv")

In [8]:
spam.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [9]:
encoder = LabelEncoder()
y = encoder.fit_transform(spam['Category'])
print(y)

[0 0 1 ... 0 0 0]


In [10]:
X = spam['Message'].values
X

array(['Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...',
       'Ok lar... Joking wif u oni...',
       "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's",
       ..., 'Pity, * was in mood for that. So...any other suggestions?',
       "The guy did some bitching but I acted like i'd be interested in buying something else next week and he gave it to us for free",
       'Rofl. Its true to its name'], dtype=object)

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [12]:
print(X_train.shape, X_test.shape)

(3900,) (1672,)


In [13]:
vectorizer = CountVectorizer()

In [14]:
vectorizer.fit(X_train) #somente cria o modelo

CountVectorizer()

In [15]:
X_train = vectorizer.transform(X_train)

In [16]:
X_test = vectorizer.transform(X_test)

In [17]:
print(X_train.toarray())

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


# Implementação da rede neural utilizando Keras

As funções de ativação são fundamentais para redes neurais, pois introduzem não-linearidade, permitindo que a rede aprenda padrões complexos.

O Dropout é uma técnica de regularização usada em redes neurais para reduzir overfitting. Ele funciona desativando aleatoriamente neurônios durante o treinamento.

Sem dropout, redes neurais muito profundas podem:<br>
❌ Memorizar os dados de treino (overfitting)<br>
❌ Tornar-se dependentes de poucos neurônios específicos<br>
❌ Generalizar mal para dados novos
<br><br>
Com dropout:<br>
✅ A rede aprende padrões mais robustos<br>
✅ Melhora a capacidade de generalização<br>
✅ Reduz a dependência excessiva de neurônios específicos

In [18]:
print(X_train.shape[1])

7262


In [19]:
model = Sequential()
model.add(Dense(units=10,
                activation="relu",
                 #necessário na primeira camada. Cada neurônio representa uma feature do conjunto de dados.
                input_dim=X_train.shape[1]
                ))
model.add(Dropout(0.1))
model.add(Dense(units=8, activation="relu"))
model.add(Dropout(0.1))
model.add(Dense(units=1, activation="sigmoid"))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [20]:
model.compile(loss='mean_squared_error',
              optimizer="adam",
              metrics=["accuracy", "precision"])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 10)                  │          72,630 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 10)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 8)                   │              88 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 8)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │               9 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 72,727 (284.09 KB)

 Trainable params: 72,727 (284.09 KB)

 Non-trainable params: 0 (0.00 B)

In [22]:
model.fit(X_train, y_train,
          #quantidade de vezes que os dados passarão pela rede(muito alto, vai levar muito tempo, muito baixo, o modelo pode não aprender)
          epochs=30,
          batch_size=10, #a cada quantos registros a rede será atualizada
          validation_data=(X_test, y_test), #passa também os dados de teste para a rede neural ir corrigindo
          verbose=True
          )

Epoch 1/30
390/390 ━━━━━━━━━━━━━━━━━━━━ 167s 421ms/step - accuracy: 0.8391 - loss: 0.1650 - precision: 0.4245 - val_accuracy: 0.9844 - val_loss: 0.0189 - val_precision: 0.9626
Epoch 2/30
390/390 ━━━━━━━━━━━━━━━━━━━━ 26s 67ms/step - accuracy: 0.9870 - loss: 0.0173 - precision: 0.9880 - val_accuracy: 0.9880 - val_loss: 0.0108 - val_precision: 0.9766
Epoch 3/30
390/390 ━━━━━━━━━━━━━━━━━━━━ 28s 34ms/step - accuracy: 0.9949 - loss: 0.0065 - precision: 0.9972 - val_accuracy: 0.9880 - val_loss: 0.0102 - val_precision: 0.9811
Epoch 4/30
390/390 ━━━━━━━━━━━━━━━━━━━━ 12s 31ms/step - accuracy: 0.9985 - loss: 0.0027 - precision: 1.0000 - val_accuracy: 0.9886 - val_loss: 0.0098 - val_precision: 0.9767
Epoch 5/30
390/390 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - accuracy: 0.9978 - loss: 0.0023 - precision: 0.9993 - val_accuracy: 0.9892 - val_loss: 0.0091 - val_precision: 0.9858
Epoch 6/30
390/390 ━━━━━━━━━━━━━━━━━━━━ 6s 15ms/step - accuracy: 0.9986 - loss: 0.0018 - precision: 0.9991 - val_accuracy: 0.9898

In [23]:
loss, accuracy, precision = model.evaluate(X_test, y_test)
print("Loss: ", loss)
print("Accuracy: ", accuracy)
print("Precision: ", precision)

53/53 ━━━━━━━━━━━━━━━━━━━━ 10s 187ms/step - accuracy: 0.9898 - loss: 0.0096 - precision: 0.9810
Loss:  0.007225021719932556
Accuracy:  0.9922248721122742
Precision:  0.9906976819038391


In [24]:
prevs = model.predict(X_test)
prevs

53/53 ━━━━━━━━━━━━━━━━━━━━ 6s 111ms/step


array([[3.2079647e-10],
       [3.5496880e-10],
       [1.2314266e-12],
       ...,
       [1.0480536e-10],
       [9.9999571e-01],
       [2.5577478e-11]], dtype=float32)

In [27]:
#se for True é spam
prevs = prevs > 0.5
prevs

array([[False],
       [False],
       [False],
       ...,
       [False],
       [ True],
       [False]])

In [28]:
cm = confusion_matrix(y_test, prevs)
print(cm)

[[1446    2]
 [  11  213]]
